In [1]:
import os
import time
import numpy as np
import renom as rm
from tqdm import tqdm
import matplotlib.pyplot as plt

from renom_img.api.model.vgg import VGG16
from renom_img.api.utility.distributor.distributor import ImageDistributor
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import *
from renom_img.api.utility.load import parse_xml_detection
from renom_img.api.utility.misc.display import draw_box

from renom.cuda import set_cuda_active
set_cuda_active(True)

In [2]:
if not os.path.exists("101_ObjectCategories"):
    !wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
    !tar zxfv 101_ObjectCategories.tar.gz

In [3]:
image_caltech101 = "101_ObjectCategories"
class_map = sorted(os.listdir(image_caltech101))[1:]

image_path_list = []
label_list = []

for i, c in enumerate(class_map):
    root_path = os.path.join(image_caltech101, c)
    img_files = os.listdir(root_path)
    image_path_list.extend([os.path.join(root_path, path) for path in img_files])
    label_list += [i]*len(img_files)

N = len(image_path_list)
perm = np.random.permutation(N)
train_N = int(N*0.8)

train_image_path_list = [image_path_list[p] for p in perm[:train_N]]
train_label_path_list = [label_list[p] for p in perm[:train_N]]

valid_image_path_list = [image_path_list[p] for p in perm[train_N:]]
valid_label_path_list = [label_list[p] for p in perm[train_N:]]

print("Dataset size")
print("  Train:{}".format(len(train_image_path_list)))
print("  Valid:{}\n".format(len(valid_image_path_list)))

print("Class list")
for i, name in enumerate(class_map):
    print("  {:02d} {}".format(i, name))

Dataset size
  Train:6941
  Valid:1736

Class list
  00 Faces
  01 Faces_easy
  02 Leopards
  03 Motorbikes
  04 accordion
  05 airplanes
  06 anchor
  07 ant
  08 barrel
  09 bass
  10 beaver
  11 binocular
  12 bonsai
  13 brain
  14 brontosaurus
  15 buddha
  16 butterfly
  17 camera
  18 cannon
  19 car_side
  20 ceiling_fan
  21 cellphone
  22 chair
  23 chandelier
  24 cougar_body
  25 cougar_face
  26 crab
  27 crayfish
  28 crocodile
  29 crocodile_head
  30 cup
  31 dalmatian
  32 dollar_bill
  33 dolphin
  34 dragonfly
  35 electric_guitar
  36 elephant
  37 emu
  38 euphonium
  39 ewer
  40 ferry
  41 flamingo
  42 flamingo_head
  43 garfield
  44 gerenuk
  45 gramophone
  46 grand_piano
  47 hawksbill
  48 headphone
  49 hedgehog
  50 helicopter
  51 ibis
  52 inline_skate
  53 joshua_tree
  54 kangaroo
  55 ketch
  56 lamp
  57 laptop
  58 llama
  59 lobster
  60 lotus
  61 mandolin
  62 mayfly
  63 menorah
  64 metronome
  65 minaret
  66 nautilus
  67 octopus
  68 okapi


In [4]:
model = VGG16(class_map, load_weight=True, train_whole_network=False)

In [5]:
model.fit(train_image_path_list, train_label_path_list)

Epoch:000 Train Loss:  nan:   4%|▍         | 17/434 [00:14<03:23,  2.05it/s] 

KeyboardInterrupt: 